### Data collection for various metrics:

Data Collection for Safety and Stability Metrics (Ignore the efficiency metrics even though they are displayed). 

Collection performed in 3 stages

    A. Data collection for Safety and CAV (Controller Acceleration Variation) 
    B. Data collevtion for WAR (Wave Attenuation Ratio)
    C. Data collection for Stability Plot

### Notes:

- Desired velocity change for FS May need monitoring (May not be set for all possible lengths)
- PI can have failures, so have to turn ON render and manually check
- LACC needs a different shock time (for no shock change time at eval)
- Generated data will be saved in the folder `test_time_rollout`
- We apply shocks after the systems have been stabilized
- Default shock times are between 8000 and 11000 unless explicitly specified otherwise
- Difference: For stability plot, the length is 270 whereas for WAR, the length is 260
- For WAR num_rollours is set to 1

### Penetration rates
- num_controlled = 1 (5%), 4(20%), 9(40%), 13(16%) 


### A and B together: (Data collection for Safety and CAV) + (Data collection for WAR)

In [1]:
# set parameters here
NUM_ROLLOUTS = 5
LENGTH = 260


### 1. (100%) IDM 

In [ ]:
!python classic.py --method idm --length $LENGTH --gen_emission --num_rollouts $NUM_ROLLOUTS --shock --render

In [ ]:
!python eval_metrics.py --method idm --save_plots

In [ ]:
# IDM does not need WAR, un-applicable

### 2. (5%) Single Vehicle Control

__FS__

In [ ]:
!python classic.py --method fs --length $LENGTH --gen_emission --num_rollouts $NUM_ROLLOUTS --shock --render --num_controlled 1

In [ ]:
!python eval_metrics.py --method fs --save_plots

In [ ]:
# Data for WAR
!python classic.py --method fs --stability --length $LENGTH --num_rollouts 1 --gen_emission --render --num_controlled 1

In [ ]:
# WAR
!python eval_plots.py --method fs 

__PIwS__
- This requires verifying if the particular run was `SUCCESS` or `FAIL`
- i.e., the controller may fail to stabilize the system

In [ ]:
# For CAV (first: get the data)
!python classic.py --method piws --length $LENGTH --gen_emission --num_rollouts $NUM_ROLLOUTS --shock --render --num_controlled 1

# Stable or not before shocks were applied
# 1. Stable 
# 2. Stable
# 3. Stable
# 4. Stable
# 5. Stable


In [ ]:
# use the eval_metrics code
!python eval_metrics.py --method pi --save_plots

In [ ]:
# Data for WAR (Check if it stabilized or not)
!python classic.py --method piws --stability --length $LENGTH --num_rollouts 1 --gen_emission --render --num_controlled 1

# Stable or not before shocks were applied
# 1. 
# 2. 
# 3. 
# 4. 
# 5. 


In [ ]:
!python eval_plots.py --method pi 

#### BCM

In [ ]:
# For CAV (first get the data)
!python classic.py --method bcm --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled 1

In [ ]:
# use the eval_metrics code
!python eval_metrics.py --method bcm --save_plots

In [ ]:
# No WAR. BCM is stable.

#### LACC

In [ ]:
# For CAV (first get the data)
!python classic.py --method lacc --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --shock_start_time 11400 --shock_end_time 15000 --num_controlled 1


In [ ]:
# use the eval_metrics code
!python eval_metrics.py --method lacc --start_time 11400 --end_time 15000 --save_plots

__Wu et. al.__

In [ ]:
# For CAV (first get the data)
!python test_rllib.py ./Wu_et_al/Trained_policies/5_percent/PPO_WaveAttenuationPOEnv-v0_25b5cb6e_2022-01-26_10-58-12e9f4i3ao 50 \
--method wu --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled 1


In [ ]:
# For CAV
!python eval_metrics.py --method wu --save_plots

In [ ]:
# Data for WAR
!python test_rllib.py ./Wu_et_al/Trained_policies/5_percent/PPO_WaveAttenuationPOEnv-v0_25b5cb6e_2022-01-26_10-58-12e9f4i3ao 50\
--method wu --shock --stability --length $LENGTH --gen_emission --num_rollouts 1 --render --num_controlled 1


In [ ]:
!python eval_plots.py --method wu 

__Ours (1x)__
- The local zone is set to 50m at training. And it is set to 50m here as well.

In [ ]:
# For CAV (Safety + Stability)
!python test_rllib.py ./Ours/Trained_policies/5_percent/PPO_DensityAwareRLEnv-v0_5dfded14_2024-02-07_09-58-351869u4p3 168\
--method ours --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled 1


In [ ]:
# Only look at safety and stability
!python eval_metrics.py --method ours --save_plots

In [ ]:
# For CAV (Efficiency)

!python test_rllib.py ./Ours/Trained_policies/5_percent/PPO_DensityAwareRLEnv-v0_6106dcf6_2024-02-08_12-42-07_7vnf1n4 200\
--method ours --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled 1

In [ ]:
# Only look at Efficiency (Wipe out the previous data)
!python eval_metrics.py --method ours --save_plots

In [ ]:
# Data for WAR (Take the Safety + Stability Policy)
!python test_rllib.py ./Ours/Trained_policies/5_percent/PPO_DensityAwareRLEnv-v0_5dfded14_2024-02-07_09-58-351869u4p3 168\
--method ours --shock --stability --length $LENGTH --gen_emission --num_rollouts 1 --render --num_controlled 1


In [ ]:
!python eval_plots.py --method ours

### 3. (20%) Multiple Vehicle Control

### FS

In [ ]:
!python classic.py --method fs --length $LENGTH --gen_emission --num_rollouts $NUM_ROLLOUTS --shock --render --num_controlled 4


In [ ]:
# use the eval_metrics code
!python eval_metrics.py --method fs --save_plots

In [ ]:
# Data for WAR
!python classic.py --method fs --stability --length $LENGTH --num_rollouts 4 --gen_emission --render --num_controlled 4


In [ ]:
# WAR
!python eval_plots.py --method fs 

__PIwS__

In [ ]:
!python classic.py --method piws --length $LENGTH --gen_emission --num_rollouts $NUM_ROLLOUTS --shock --render --num_controlled 4


In [ ]:
!python eval_metrics.py --method pi --save_plots

In [ ]:
# Data for WAR (Check if it stabilized or not)
!python classic.py --method piws --stability --length $LENGTH --num_rollouts 1 --gen_emission --render --num_controlled 4


In [ ]:
!python eval_plots.py --method pi 

__BCM__

In [ ]:
# For CAV (first get the data)
!python classic.py --method bcm --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled 4

In [ ]:
# use the eval_metrics code
!python eval_metrics.py --method bcm --save_plots

In [ ]:
# Data for WAR
!python classic.py --method bcm --stability --length $LENGTH --gen_emission --num_rollouts $NUM_ROLLOUTS --render --num_controlled 4


In [ ]:
!python eval_plots.py --method bcm 

__LACC__

In [ ]:
# For CAV (first get the data)
!python classic.py --method lacc --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled 4 --shock_start_time 11400 --shock_end_time 15000

In [ ]:
# use the eval_metrics code
!python eval_metrics.py --method lacc --start_time 11400 --end_time 15000 --save_plots

In [ ]:
# Data for WAR
!python classic.py --method lacc --stability --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --render --num_controlled 4 --shock_start_time 11400 --shock_end_time 15000


In [ ]:
!python eval_plots.py --method lacc --start_time 11400 --end_time 15000 --propogate_time 200

__Ours (20%)__

- The local zone of __leader__ is set to 50m at training. It is kept at 50m here as well.

In [ ]:
#### Safety + Stability ####
# For CAV (first get the data). Copy the exp_configs to a folder where test_rllib.py is located
# Make sure efficiency is set to False in controller
!python test_rllib.py ./Ours/Trained_policies/20_percent/PPO_MultiAgentDensityAwareRLEnv-v1_afca7452_2024-02-11_14-35-18pt1771a1 140\
--method ours4x --length $LENGTH --num_rollouts 20 --gen_emission --shock --render --num_controlled 4


2024-02-12 10:00:58,740	WARNING services.py:597 -- setpgrp failed, processes may not be cleaned up properly: [Errno 1] Operation not permitted.
2024-02-12 10:00:58,740	INFO resource_spec.py:216 -- Starting Ray with 25.49 GiB memory available for workers and up to 12.76 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2024-02-12 10:01:00,570	INFO trainer.py:371 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
2024-02-12 10:01:00,576	INFO trainer.py:512 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2024-02-12 10:01:03,653	WARNING util.py:45 -- Install gputil for GPU system monitoring.
2024-02-12 10:01:03,705	INFO trainable.py:346 -- Restored from checkpoint: ./Ours/Trained_policies/20_percent/PPO_MultiAgentDensityAwareRLEnv-v1_afca7452_2024-02-11_14-35-18pt1771a1/checkpoint_140/checkpoint-140
2024-02-12 10:01:03,705	INFO trainabl

Step = 6582, Shock params: -2.175281412618169, 2.0, 15 applied to vehicle human_2_12

Step = 6583, Shock params: -2.175281412618169, 2.0, 15 applied to vehicle human_2_12

Step = 6777, Shock params: -1.9523591616308036, 0.6, 15 applied to vehicle human_2_11

Step = 6778, Shock params: -1.9523591616308036, 0.6, 15 applied to vehicle human_2_11

Step = 6779, Shock params: -1.9523591616308036, 0.6, 15 applied to vehicle human_2_11

Step = 6780, Shock params: -1.9523591616308036, 0.6, 15 applied to vehicle human_2_11

Step = 6781, Shock params: -1.9523591616308036, 0.6, 15 applied to vehicle human_2_11

Step = 6782, Shock params: -1.9523591616308036, 0.6, 15 applied to vehicle human_2_11

Step = 6990, Shock params: -2.338851059285158, 1.0, 15 applied to vehicle human_2_5

Step = 6991, Shock params: -2.338851059285158, 1.0, 15 applied to vehicle human_2_5

Step = 6992, Shock params: -2.338851059285158, 1.0, 15 applied to vehicle human_2_5

Step = 6993, Shock params: -2.338851059285158, 1.0,

Step = 8066, Shock params: -2.9212710382763944, 2.2, 15 applied to vehicle human_2_8

Step = 8067, Shock params: -2.9212710382763944, 2.2, 15 applied to vehicle human_2_8

Step = 8068, Shock params: -2.9212710382763944, 2.2, 15 applied to vehicle human_2_8

Step = 8069, Shock params: -2.9212710382763944, 2.2, 15 applied to vehicle human_2_8

Step = 8070, Shock params: -2.9212710382763944, 2.2, 15 applied to vehicle human_2_8

Step = 8071, Shock params: -2.9212710382763944, 2.2, 15 applied to vehicle human_2_8

Step = 8072, Shock params: -2.9212710382763944, 2.2, 15 applied to vehicle human_2_8

Step = 8073, Shock params: -2.9212710382763944, 2.2, 15 applied to vehicle human_2_8

Step = 8074, Shock params: -2.9212710382763944, 2.2, 15 applied to vehicle human_2_8

Step = 8075, Shock params: -2.9212710382763944, 2.2, 15 applied to vehicle human_2_8

Step = 8076, Shock params: -2.9212710382763944, 2.2, 15 applied to vehicle human_2_8

Step = 8268, Shock params: -1.4269187067431142, 0.4, 1

Step = 6006, Shock params: -2.149543580761971, 1.9, 19 applied to vehicle human_2_6

Step = 6007, Shock params: -2.149543580761971, 1.9, 19 applied to vehicle human_2_6

Step = 6008, Shock params: -2.149543580761971, 1.9, 19 applied to vehicle human_2_6

Step = 6009, Shock params: -2.149543580761971, 1.9, 19 applied to vehicle human_2_6

Step = 6010, Shock params: -2.149543580761971, 1.9, 19 applied to vehicle human_2_6

Step = 6011, Shock params: -2.149543580761971, 1.9, 19 applied to vehicle human_2_6

Step = 6012, Shock params: -2.149543580761971, 1.9, 19 applied to vehicle human_2_6

Step = 6013, Shock params: -2.149543580761971, 1.9, 19 applied to vehicle human_2_6

Step = 6014, Shock params: -2.149543580761971, 1.9, 19 applied to vehicle human_2_6

Step = 6015, Shock params: -2.149543580761971, 1.9, 19 applied to vehicle human_2_6

Step = 6016, Shock params: -2.149543580761971, 1.9, 19 applied to vehicle human_2_6

Step = 6164, Shock params: -1.1692809251832048, 0.2, 19 applied t

Step = 7329, Shock params: -2.4417742361762027, 0.6, 19 applied to vehicle human_2_6

Step = 7330, Shock params: -2.4417742361762027, 0.6, 19 applied to vehicle human_2_6

Step = 7331, Shock params: -2.4417742361762027, 0.6, 19 applied to vehicle human_2_6

Step = 7492, Shock params: -1.2689779343861165, 0.6, 19 applied to vehicle human_2_7

Step = 7493, Shock params: -1.2689779343861165, 0.6, 19 applied to vehicle human_2_7

Step = 7494, Shock params: -1.2689779343861165, 0.6, 19 applied to vehicle human_2_7

Step = 7495, Shock params: -1.2689779343861165, 0.6, 19 applied to vehicle human_2_7

Step = 7496, Shock params: -1.2689779343861165, 0.6, 19 applied to vehicle human_2_7

Step = 7497, Shock params: -1.2689779343861165, 0.6, 19 applied to vehicle human_2_7

Step = 7658, Shock params: -1.3571163034606857, 1.2, 19 applied to vehicle human_2_13

Step = 7659, Shock params: -1.3571163034606857, 1.2, 19 applied to vehicle human_2_13

Step = 7660, Shock params: -1.3571163034606857, 1.2,

...

Step = 8496, Shock params: -1.2315371979868948, 1.2, 13 applied to vehicle human_2_10

Step = 8497, Shock params: -1.2315371979868948, 1.2, 13 applied to vehicle human_2_10

Step = 8498, Shock params: -1.2315371979868948, 1.2, 13 applied to vehicle human_2_10

Step = 8499, Shock params: -1.2315371979868948, 1.2, 13 applied to vehicle human_2_10

Round 2, Return: {'follower': [-464057.0760781567]} for agent follower

-----------------------
ring length: 260
v_max: 5.139779427502188
-----------------------
/mnt/c/Users/09_gi/Desktop/Beyond-Simulated-Drivers/ring/test_time_rollout/ours4x/density_aware_multiagent_ring_20240212-1008201707750500.9364872-0_emission.csv /mnt/c/Users/09_gi/Desktop/Beyond-Simulated-Drivers/ring/test_time_rollout/ours4x
2024-02-12 10:11:48,379	ERROR worker.py:679 -- Calling ray.init() again after it has already been called.
2024-02-12 10:11:51,461	WARNING util.py:45 -- Install gputil for GPU system monitoring.
2024-02-12 10:11:51,509	INFO trainable.py:346 -- Rest

Step = 6380, Shock params: -2.4934227395742457, 2.0, 18 applied to vehicle human_2_0

Step = 6381, Shock params: -2.4934227395742457, 2.0, 18 applied to vehicle human_2_0

Step = 6382, Shock params: -2.4934227395742457, 2.0, 18 applied to vehicle human_2_0

Step = 6383, Shock params: -2.4934227395742457, 2.0, 18 applied to vehicle human_2_0

Step = 6384, Shock params: -2.4934227395742457, 2.0, 18 applied to vehicle human_2_0

Step = 6385, Shock params: -2.4934227395742457, 2.0, 18 applied to vehicle human_2_0

Step = 6386, Shock params: -2.4934227395742457, 2.0, 18 applied to vehicle human_2_0

Step = 6387, Shock params: -2.4934227395742457, 2.0, 18 applied to vehicle human_2_0

Step = 6388, Shock params: -2.4934227395742457, 2.0, 18 applied to vehicle human_2_0

Step = 6389, Shock params: -2.4934227395742457, 2.0, 18 applied to vehicle human_2_0

Step = 6390, Shock params: -2.4934227395742457, 2.0, 18 applied to vehicle human_2_0

Step = 6391, Shock params: -2.4934227395742457, 2.0, 1

Step = 7448, Shock params: -1.7566809075101257, 1.4, 18 applied to vehicle human_2_1

Step = 7449, Shock params: -1.7566809075101257, 1.4, 18 applied to vehicle human_2_1

Step = 7612, Shock params: -1.6011793495029183, 1.2, 18 applied to vehicle human_2_4

Step = 7613, Shock params: -1.6011793495029183, 1.2, 18 applied to vehicle human_2_4

Step = 7614, Shock params: -1.6011793495029183, 1.2, 18 applied to vehicle human_2_4

Step = 7615, Shock params: -1.6011793495029183, 1.2, 18 applied to vehicle human_2_4

Step = 7616, Shock params: -1.6011793495029183, 1.2, 18 applied to vehicle human_2_4

Step = 7617, Shock params: -1.6011793495029183, 1.2, 18 applied to vehicle human_2_4

Step = 7618, Shock params: -1.6011793495029183, 1.2, 18 applied to vehicle human_2_4

Step = 7619, Shock params: -1.6011793495029183, 1.2, 18 applied to vehicle human_2_4

Step = 7620, Shock params: -1.6011793495029183, 1.2, 18 applied to vehicle human_2_4

Step = 7621, Shock params: -1.6011793495029183, 1.2, 1

Step = 5508, Shock params: -2.6758271649189194, 2.4, 10 applied to vehicle human_2_0

Step = 5509, Shock params: -2.6758271649189194, 2.4, 10 applied to vehicle human_2_0

Step = 5510, Shock params: -2.6758271649189194, 2.4, 10 applied to vehicle human_2_0

Step = 5511, Shock params: -2.6758271649189194, 2.4, 10 applied to vehicle human_2_0

Step = 5512, Shock params: -2.6758271649189194, 2.4, 10 applied to vehicle human_2_0

Step = 5513, Shock params: -2.6758271649189194, 2.4, 10 applied to vehicle human_2_0

Step = 5514, Shock params: -2.6758271649189194, 2.4, 10 applied to vehicle human_2_0

Step = 5515, Shock params: -2.6758271649189194, 2.4, 10 applied to vehicle human_2_0

Step = 5516, Shock params: -2.6758271649189194, 2.4, 10 applied to vehicle human_2_0

Step = 5517, Shock params: -2.6758271649189194, 2.4, 10 applied to vehicle human_2_0

Step = 5518, Shock params: -2.6758271649189194, 2.4, 10 applied to vehicle human_2_0

Step = 5519, Shock params: -2.6758271649189194, 2.4, 1

Step = 7830, Shock params: -2.416585285071375, 1.4, 10 applied to vehicle human_2_3

Step = 7831, Shock params: -2.416585285071375, 1.4, 10 applied to vehicle human_2_3

Step = 7832, Shock params: -2.416585285071375, 1.4, 10 applied to vehicle human_2_3

Step = 7833, Shock params: -2.416585285071375, 1.4, 10 applied to vehicle human_2_3

Step = 7834, Shock params: -2.416585285071375, 1.4, 10 applied to vehicle human_2_3

Step = 7835, Shock params: -2.416585285071375, 1.4, 10 applied to vehicle human_2_3

Step = 7836, Shock params: -2.416585285071375, 1.4, 10 applied to vehicle human_2_3

Step = 7837, Shock params: -2.416585285071375, 1.4, 10 applied to vehicle human_2_3

Step = 8156, Shock params: -2.0368789387753896, 1.0, 10 applied to vehicle human_2_17

Step = 8157, Shock params: -2.0368789387753896, 1.0, 10 applied to vehicle human_2_17

Step = 8158, Shock params: -2.0368789387753896, 1.0, 10 applied to vehicle human_2_17

Step = 8159, Shock params: -2.0368789387753896, 1.0, 10 app

In [ ]:
# Since !human is present in the code, this works seamlessly
!python eval_metrics.py --method ours4x --save_plots

In [ ]:
#### Efficiency ####
# Make sure to set the efficiency to true at test time in the controller file and have those files at exp_configs
!python test_rllib.py ./Ours/Trained_policies/20_percent/PPO_MultiAgentDensityAwareRLEnv-v1_33361ae0_2024-02-11_20-29-43f1pzfgrz 140\
--method ours4x --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled 4


In [ ]:
!python eval_metrics.py --method ours4x --save_plots

In [ ]:
# Data for WAR (taken on the Safety + Stability model)

!python test_rllib.py ./Ours/Trained_policies/20_percent/PPO_MultiAgentDensityAwareRLEnv-v1_afca7452_2024-02-11_14-35-18pt1771a1 140\
--method ours4x --shock --stability --length $LENGTH --gen_emission --num_rollouts $NUM_ROLLOUTS --render


In [ ]:
!python eval_plots.py --method ours4x

### 4. (40%) Multiple Vehicle Control

__Ours (9x)__
- No stability plot is drawn for this one but still need WAR
- The local zone of __leader__ is set to 50m at training, Change it to 55m here.

In [ ]:
# Local zone is kept at 55m 

# For CAV (first get the data)
!python test_rllib.py ./Ours/Trained_policies/Multi_agent_9x/PPO_MultiAgentDensityAwareRLEnv-v1_40834376_2023-07-30_19-43-22ly9k9abp 116\
--method ours9x --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render

In [ ]:
!python eval_metrics.py --method ours9x --save_plots

In [ ]:
# Data for WAR
!python test_rllib.py ./Ours/Trained_policies/Multi_agent_9x/PPO_MultiAgentDensityAwareRLEnv-v1_40834376_2023-07-30_19-43-22ly9k9abp 116\
--method ours9x --shock --stability --length $LENGTH --gen_emission --num_rollouts $NUM_ROLLOUTS --render

In [ ]:
!python eval_plots.py --method ours9x 

## C. Data collection for Stability Plot
- Execute the `eval_plots.py` code as well because this will create the stability plots (sometimes manual inspection is required)

In [ ]:
LENGTH_STABILITY = 270 #m
NUM_ROLLOUTS_STABILITY = 1

### IDM

In [ ]:
# This is for plot
# Stability shock is applied to IDM when the system is stable and SGT is not formed yet
# 150 is pretty early-on
!python classic.py --method idm --length $LENGTH_STABILITY --gen_emission --stability --shock_start_time 150 --shock_end_time 3710 --render

In [ ]:
# IDM is unstable (So there is no wave attenuation) - WAR does not apply
# But do we still need a value for reference (WAR = 0)
!python eval_plots.py --method idm --start_time 150 --end_time 3710 

### FS

In [ ]:
!python classic.py --method fs --stability --length $LENGTH_STABILITY --num_rollouts $NUM_ROLLOUTS_STABILITY --gen_emission --render

In [ ]:
!python eval_plots.py --method fs 

### PIwS

In [ ]:
# Data for WAR (Check if it stabilized or not)
!python classic.py --method piws --stability --length $LENGTH_STABILITY --num_rollouts $NUM_ROLLOUTS_STABILITY --gen_emission --render
# Check if was stable or not before the shock was applied

In [ ]:
!python eval_plots.py --method pi 

### RL+ LO (Wu et al.)

In [ ]:
!python test_rllib.py ./Wu_et_al/Trained_policies/PPO_WaveAttenuationPOEnv-v0_25b5cb6e_2022-01-26_10-58-12e9f4i3ao 50\
--method wu --shock --stability --length $LENGTH_STABILITY --gen_emission --num_rollouts $NUM_ROLLOUTS_STABILITY --render


In [ ]:
!python eval_plots.py --method wu 

### Ours (1x)
- Set local zone to 35m in the file before executing the code below

In [ ]:
# This data is taken with the local zone set to 35m
!python test_rllib.py ./Ours/Trained_policies/Single_agent/PPO_DensityAwareRLEnv-v0_2355c52c_2023-07-21_18-27-20qktx7e8o 290\
--method ours --shock --stability --length $LENGTH_STABILITY --gen_emission --num_rollouts $NUM_ROLLOUTS_STABILITY --render

In [ ]:
!python eval_plots.py --method ours

### BCM

In [ ]:
!python classic.py --method bcm --stability --length $LENGTH_STABILITY --num_rollouts $NUM_ROLLOUTS_STABILITY --gen_emission --num_rollouts $NUM_ROLLOUTS_WAR --render


In [ ]:
!python eval_plots.py --method bcm 

### Ours (4x)
- Set Local zone to 40m in the file before executing the code below 

In [ ]:
# This data is taken with the local zone set to 40m
!python test_rllib.py ./Ours/Trained_policies/Multi_agent_4x/PPO_MultiAgentDensityAwareRLEnv-v1_0fac2f26_2023-07-31_09-46-415e_f62_x 48\
--method ours4x --shock --stability --length $LENGTH_STABILITY --gen_emission --num_rollouts $NUM_ROLLOUTS_STABILITY --render


In [ ]:
!python eval_plots.py --method ours4x

### LACC

In [ ]:
!python classic.py --method lacc --stability --length $LENGTH_STABILITY --num_rollouts $NUM_ROLLOUTS_STABILITY --gen_emission --shock_start_time 11400 --shock_end_time 15000 --render

In [ ]:
!python eval_plots.py --method lacc --start_time 11400 --end_time 15000 --propogate_time 200

### Ours (9x)
- This not included in the stability plot